In [1]:
import sqlite3
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

# sqlitebrowser
# create diagram https://dbdiagram.io/d

# Create sql from csv

In [2]:
genstudio = pd.read_csv('./data/genstudio.csv', dtype={'Position':str})
metadata = pd.read_csv('./data/metadata.csv')

#print(genstudio.shape, metadata.shape)

In [3]:
def get_scheme_df(df):
    '''
    Create SQL scheme from DataFrame colnames.
    Return SQL scheme and list with processed colnames.
    '''
    col_names, types = df.columns, df.dtypes
    scheme_df = ['row_id INTEGER PRIMARY KEY']
    new_names = ['row_id']


    for name, contain in zip(col_names[1:], types[1:]):
        name = name.replace("-", "").replace("  ", " ").replace(" ", "_")
        new_names.append(name)

        if 'int' in str(contain):
            scheme_df.append(' '.join([name, 'INTEGER']))
        
        elif 'float' in str(contain):
            scheme_df.append(' '.join([name, 'REAL']))
        
        else:
            scheme_df.append(' '.join([name, 'TEXT']))
            
    scheme_df = ', '.join(scheme_df)

    return scheme_df, new_names

In [4]:
scheme_genstudio, new_names = get_scheme_df(genstudio)
genstudio.columns = new_names

scheme_metadata, new_names = get_scheme_df(metadata)
metadata.columns = new_names

In [5]:
def bid_DB(genstudio, metadata, scheme_genstudio, scheme_metadata, path='./data/bid_DB.db'):
    
    connection = sqlite3.connect(path)

    connection.execute('''CREATE TABLE IF NOT EXISTS SNP_data (
    Sample_ID TEXT, 
    SNP_Name TEXT, 
    SNP TEXT, 
    Position TEXT, 
    Chr TEXT,
    sex TEXT)''')

    connection.execute(f'''CREATE TABLE IF NOT EXISTS metadata ({scheme_metadata},
                        FOREIGN KEY (sex) REFERENCES SNP_data (sex))''')

    connection.execute(f'''CREATE TABLE IF NOT EXISTS genstudio ({scheme_genstudio},
    FOREIGN KEY (Sample_ID) REFERENCES metadata (dna_chip_id),
    FOREIGN KEY (Sample_ID) REFERENCES metadata (Sample_ID),
    FOREIGN KEY (SNP_Name) REFERENCES metadata (SNP_Name),
    FOREIGN KEY (SNP) REFERENCES metadata (SNP),
    FOREIGN KEY (Position) REFERENCES metadata (Position),
    FOREIGN KEY (Chr) REFERENCES metadata (Chr))''')

    connection.commit()

    genstudio.to_sql('genstudio', connection, if_exists='append', index=False)
    metadata.to_sql('metadata', connection, if_exists='append', index=False)

    connection.close()

In [6]:
path='./data/bid_DB.db'
bid_DB(genstudio, metadata, scheme_genstudio, scheme_metadata, path)

# Selection

In [7]:
connection = sqlite3.connect('./data/bid_DB.db')

query = '''SELECT Sample_ID, SNP, Chr
           FROM genstudio
           WHERE SNP = '[A/G]'
'''

res = connection.execute(query)

print(len(res.fetchall()))

connection.close()

855870


## Add selection to DB

In [8]:
connection = sqlite3.connect('./data/bid_DB.db')

query = '''INSERT INTO SNP_data (Sample_ID, SNP_Name, SNP, Position, Chr, sex)
SELECT genstudio.Sample_ID, genstudio.SNP_Name, genstudio.SNP, genstudio.Position, genstudio.Chr, 
metadata.sex 
FROM genstudio, metadata 
WHERE metadata.dna_chip_id = genstudio.Sample_ID'''

connection.execute(query)
connection.commit()
connection.close()

# Create and add from web

In [2]:
def get_genes():
    
    GENE_URL = 'https://genomics.senescence.info/genes/allgenes.php'
    
    resp = requests.get(GENE_URL)
    soup = BeautifulSoup(resp.content, 'lxml')
    
    genes_list = []
    
    table = soup.find("table", {"class":"results-table"} )
    rows = table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        
        if cols:    
            genes_list.append(cols[0])
    
    return genes_list


def snp_search(gene_name):
    '''
    Search first 20 SNP in ncbi snp database. Return soup object.
    '''
    SNP_URL = 'https://www.ncbi.nlm.nih.gov/snp/'
    query_str = {'term': gene_name + '[Gene Name]'}
    
    resp = requests.get(SNP_URL, params=query_str)
    soup = BeautifulSoup(resp.content, 'lxml')
    
    return soup


def parse_soup(soup):
    '''
    Extract SNP ID from soup.
    Return list with rs-id.
    '''
    rs_id = []
    
    for result in soup.find_all('div', {'class': 'rprt'}):
        rs_id.append(result.find('div', {'class': 'supp'}).find('a').text)
        
    return rs_id


def get_id_list(genes_list):
    '''
    Search first 20 SNP in ncbi snp database and extract SNP ID from soup.
    Wait 5 seconds between get requests.
    Return lists with SNP ID`s for all genes.
    '''
    id_list = []
    
    for gene_name in genes_list:
        time.sleep(1)
        
        soup = snp_search(str(gene_name))
        rs_id = parse_soup(soup)

        if rs_id:
            id_list.append(rs_id)
            
    return id_list


In [3]:
def add_to_sql(snp_name, position, chromosome, alleles, db_build, assembly_build,
               clin_sig, n_publications):
    '''
    Open connection to SNP_human DB, add some info to it and close connection.
    Return nothing.
    '''
    connection = sqlite3.connect('./data/SNP_human.db')
    
    query_1 = '''INSERT INTO SNP_data (SNP_Name, Alleles, Position, Chr, dbSNP_Build, Assembly_Build)
                                       VALUES (?, ?, ?, ?, ?, ?)'''
    
    connection.execute(query_1, [snp_name, alleles, position, chromosome, db_build, assembly_build])
    
    query_2 = '''INSERT INTO clin_SNP (SNP_Name, Clinical_Significance, N_Publications)
                                       VALUES (?, ?, ?)'''
    
    connection.execute(query_2, [snp_name, clin_sig, n_publications]) 
    
    connection.commit()
    connection.close()


def get_snp_info(red_id_list):
    '''
    Get info about SNP from soup object and add it to the SQL DB.
    Wait 5 seconds between get requests.
    Return 'DONE' when done.
    '''
    i = 1
    
    for snp_name in red_id_list:
        
        time.sleep(1)
        
        req_url = f'https://www.ncbi.nlm.nih.gov/snp/{snp_name}#clinical_significance'

        resp = requests.get(req_url, params={'horizontal_tab': 'true'})
        soup = BeautifulSoup(resp.content, 'lxml')

        db_build = soup.find('div', {'class': 'accession usa-width-one-third'}).text.split()[2]

        snp_info = soup.find_all('dl', {'class': 'usa-width-one-half'})
        dd_info = snp_info[0].find_all('dd')
        build_chr_pos = dd_info[1].find_all('span') 

        assembly_build = build_chr_pos[1].text.strip()
        position = build_chr_pos[0].text.split(':')[1]
        chromosome = build_chr_pos[0].text.split(':')[0]

        alleles = ''.join(dd_info[2].text.split())
    
        clin_sig = snp_info[1].find('dd').text.strip()
        
        publ_info = snp_info[1].find('a', {'id': 'snp_pub_count'})
        
        if publ_info:
            n_publications = publ_info.text.split()[0]
        else:
            n_publications=0
        
        add_to_sql(snp_name,
                   position,
                   chromosome,
                   alleles,
                   db_build,
                   assembly_build,
                   clin_sig,
                   n_publications)

    return 'DONE'


def create_SNP_db():
    '''
    Create DB with two tables with info about human genes SNP.
    Gets information about the name of human genes from the "genomics.senescence.info" site.
    When searching for SNPs for each gene in SNP NCBI, returns no more than 20 SNPs.
    Write found SNPs to the SNP_ids.txt file.
    Search info about each SNP in SNP NCBI.
    Add info about found SNPs to SNP_human.db.
    Return DONE when done.
    
    '''
    
    connection = sqlite3.connect('./data/SNP_human.db')

    connection.execute('''CREATE TABLE IF NOT EXISTS SNP_data ( 
                           SNP_Name TEXT PRIMARY KEY, 
                           Alleles TEXT, 
                           Position TEXT, 
                           Chr TEXT,
                           dbSNP_Build TEXT,
                           Assembly_Build TEXT)''')

    connection.execute('''CREATE TABLE IF NOT EXISTS clin_SNP (
                           SNP_Name TEXT,
                           Clinical_Significance TEXT,
                           N_Publications TEXT,
                           FOREIGN KEY (SNP_Name) REFERENCES SNP_data (SNP_Name))''')

    connection.commit()
    connection.close()
    #Done in file
    '''
    genes_list = get_genes()

    id_list = get_id_list(genes_list)
    red_id_list = list(set([j for sub in id_list for j in sub]))

    with open('./data/SNP_ids.txt', 'w') as output_file:
        for rsid in red_id_list:
            output_file.write(rsid + '\n')
    '''
    with open('./data/SNP_ids.txt') as f:
        red_id_list = f.read().splitlines()
    
    return get_snp_info(red_id_list)

In [4]:
create_SNP_db()

ConnectionError: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Max retries exceeded with url: /snp/rs144060305?horizontal_tab=true (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fa9929abb20>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

# SNP DB select

In [5]:
connection = sqlite3.connect('./data/SNP_human.db')

query = '''SELECT SNP_data.SNP_Name, SNP_data.Chr, SNP_data.Position, SNP_data.Alleles
           FROM SNP_data
           LEFT JOIN clin_SNP ON SNP_data.SNP_Name = clin_SNP.SNP_Name
           WHERE clin_SNP.Clinical_Significance = 'Reported in ClinVar' AND clin_SNP.N_Publications != '0'
'''

res = connection.execute(query)

result = res.fetchall()
connection.close()

In [6]:
result

[('rs2227931', 'chr3', '142503442 ', 'A>C/A>G'),
 ('rs1042522', 'chr17', '7676154 ', 'G>A/G>C/G>T'),
 ('rs2072454', 'chr7', '55146655 ', 'C>T'),
 ('rs751281707', 'chr4', '3074924-3074925 ', 'delGC'),
 ('rs137852507', 'chrX', '71110603 ', 'T>A/T>G'),
 ('rs587777342', 'chr16', '681520 ', 'G>A/G>T'),
 ('rs35677492', 'chr11', '34471409 ', 'G>A'),
 ('rs2233783', 'chr5', '177994296 ', 'C>A/C>G/C>T'),
 ('rs1532625', 'chr16', '56971389 ', 'C>T'),
 ('rs121918118', 'chr7', '30971183 ', 'T>A'),
 ('rs398122840', 'chr19', '48955714-48955721 ', 'del(G)7/delG/dupG'),
 ('rs2228440', 'chr5', '150117665 ', 'G>A'),
 ('rs2075606', 'chr19', '1220322 ', 'T>A/T>C'),
 ('rs1075865', 'chr3', '53189911 ', 'C>A/C>G/C>T'),
 ('rs727502948', 'chr16', '30899403 ', 'C>G'),
 ('rs80356668', 'chr11', '2160829 ', 'A>C/A>G'),
 ('rs4244610', 'chr8', '144512131 ', 'G>A'),
 ('rs3176750', 'chr9', '97675507 ', 'G>C'),
 ('rs34208922', 'chr7', '74069201-74069202 ', 'insA'),
 ('rs743581', 'chr15', '74035800 ', 'G>A/G>T'),
 ('rs289

In [7]:
connection = sqlite3.connect('./data/SNP_human.db')

query = '''SELECT Chr, COUNT(SNP_Name)
           FROM SNP_data
           GROUP BY Chr
           HAVING COUNT(SNP_Name) BETWEEN 20 AND 50
'''

res = connection.execute(query)

result = res.fetchall()
connection.close()

In [8]:
result

[('chr1', 48),
 ('chr10', 39),
 ('chr11', 47),
 ('chr12', 48),
 ('chr13', 29),
 ('chr15', 28),
 ('chr16', 33),
 ('chr2', 33),
 ('chr20', 26),
 ('chr22', 28),
 ('chr4', 44),
 ('chr5', 50),
 ('chr6', 45),
 ('chr7', 31),
 ('chr9', 29)]

In [24]:
connection = sqlite3.connect('./data/SNP_human.db')

query = '''BEGIN TRANSACTION;

           DELETE FROM SNP_data
           WHERE SNP_data.SNP_Name in (
           SELECT SNP_Name FROM clin_SNP WHERE clin_SNP.Clinical_Significance = "Not Reported in ClinVar");
           
           DELETE FROM clin_SNP
           WHERE clin_SNP.Clinical_Significance = "Not Reported in ClinVar";
           
           COMMIT'''

connection.executescript(query)
connection.close()

In [20]:
connection = sqlite3.connect('./data/SNP_human.db')

query = '''SELECT Chr, COUNT(SNP_Name)
           FROM SNP_data
           GROUP BY Chr
           HAVING COUNT(SNP_Name) BETWEEN 20 AND 50
'''

res = connection.execute(query)

result = res.fetchall()
connection.close()

In [21]:
result

[('chr1', 28),
 ('chr10', 29),
 ('chr11', 36),
 ('chr12', 36),
 ('chr15', 28),
 ('chr17', 36),
 ('chr19', 38),
 ('chr2', 21),
 ('chr3', 41),
 ('chr4', 23),
 ('chr5', 35),
 ('chr7', 25),
 ('chr8', 31),
 ('chr9', 24)]